In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
import re
import pdfplumber

In [6]:
url = 'https://www.boe.es/boe_n/dias/2024/03/08/index.php?l=N&e=L15#contenedor'         # Define la URL de la página web

response = requests.get(url)                                                            # Realiza una solicitud GET a la URL
response.status_code                                                                    # Obtiene el código de estado de la respuesta

soup = BeautifulSoup(response.text, 'html.parser')                                      # Crea un objeto BeautifulSoup a partir del HTML de la respuesta

condicion1 = soup.find('h6', text='AYUNTAMIENTO DE CORUÑA, A')                          # Busca un elemento <h6> con el texto 'AYUNTAMIENTO DE CORUÑA, A'
if condicion1 is not None:  
    print("Condición 1 encontrada: ", condicion1.text)                                  # Imprime el texto del elemento <h6> encontrado
    condicion3 = condicion1.find_next('h6')                                             # Encuentra el siguiente elemento <h6>

    html = str(response.content, 'utf-8')                                               # Convierte el contenido de la respuesta a una cadena

    start_index = html.find(str(condicion1))                                            # Encuentra el índice de inicio del texto de condicion1 en la cadena html
    end_index = html.find(str(condicion3)) + len(str(condicion3))                       # Encuentra el índice de final del texto de condicion3 en la cadena html

    sub_html = html[start_index:end_index]                                              # Extrae el subconjunto de HTML entre condicion1 y condicion3
    sub_soup = BeautifulSoup(sub_html, 'html.parser')                                   # Crea un objeto BeautifulSoup del subconjunto de HTML

    sub_text = sub_soup.get_text()                                                      # Obtiene el texto del subconjunto de HTML

    patron = r"MOVILIDAD\.\s+((?:\S+\s+){9})"                                           # Define un patrón de expresión regular para buscar "MOVILIDAD." seguido de 9 palabras

    coincidencias = re.findall(patron, sub_text)                                        # Busca todas las coincidencias del patrón en el subconjunto de HTML

    coincidencias_limpio = [texto.replace('\n','').strip() for texto in coincidencias]  # Elimina los saltos de línea y espacios en blanco de las coincidencias
    print("Patron encontrado: ", coincidencias_limpio)                                  # Imprime los textos con el patron MOVILIDAD. encontrados
    condicion2 = sub_soup.find('p', text="MOVILIDAD. " + coincidencias_limpio[0])
    print(condicion2)
    pdf_links = {}                                                                      # Diccionario para almacenar los enlaces de descarga del PDF

    for i in range(len(coincidencias_limpio)):                                          # Itera sobre las coincidencias limpias
        if i == 0:                                                                      # Si es la primera coincidencia
            pdf_links[i] = condicion2.find_next('a')                                           # Almacena el enlace en el diccionario pdf_links
        else:                                                                           # Para todas las demás coincidencias
            pdf_links[i] = pdf_links[i - 1].find_next('a')                              # Almacena el siguiente enlace en el diccionario pdf_links

    for key, value in pdf_links.items():                                                # Itera sobre el diccionario pdf_links

        pdf_url = value['href']                                                         # Obtiene la URL del enlace
        pdf_response = requests.get('https://www.boe.es/' + pdf_url)                    # Realiza una solicitud GET para descargar el PDF

        dir_path = "C:/Users/axier/Desktop/IRONHACK/FINAL_PROYECT/PDF_DOWNLOAD"         # Ruta de la carpeta de descarga
        os.makedirs(dir_path, exist_ok=True)                                            # Crea la carpeta si no existe
        with open(os.path.join(dir_path, f"pdf_boe_{key}.pdf"), 'wb') as f:             # Abre el archivo PDF para escritura binaria
            f.write(pdf_response.content)                                               # Escribe el contenido del PDF en el archivo

    print("Archivos descargados exitosamente")                                          # Muestra un mensaje de éxito
else:
    print("La condición 1 no se ha encontrado")                                         # Muestra un mensaje si no se encuentra condicion1 en el HTML

C:\Users\axier\AppData\Local\Temp\ipykernel_1660\265026630.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  condicion1 = soup.find('h6', text='AYUNTAMIENTO DE CORUÑA, A')                          # Busca un elemento <h6> con el texto 'AYUNTAMIENTO DE CORUÑA, A'
C:\Users\axier\AppData\Local\Temp\ipykernel_1660\265026630.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  condicion2 = sub_soup.find('p', text="MOVILIDAD. " + coincidencias_limpio[0])


Condición 1 encontrada:  AYUNTAMIENTO DE CORUÑA, A
Patron encontrado:  ['Anuncio de notificación de 6 de febrero de 2024.', 'Anuncio de notificación de 13 de febrero de 2024.', 'Anuncio de notificación de 20 de febrero de 2024.']
<p>MOVILIDAD. Anuncio de notificación de 6 de febrero de 2024.</p>
Archivos descargados exitosamente


In [8]:
folder_path = "C:/Users/axier/Desktop/IRONHACK/FINAL_PROYECT/PDF_DOWNLOAD"                # Establece la ruta a la carpeta que contiene los archivos PDF

all_dfs = []                                                                              # Inicializa una lista para almacenar todos los DataFrames de las tablas

for filename in os.listdir(folder_path):                                                  # Itera sobre cada archivo en la carpeta
    if filename.endswith(".pdf"):                                                         # Selecciona los archivos que terminen con .pdf
        file_path = os.path.join(folder_path, filename)                                   # Obtiene la ruta completa del archivo
        with pdfplumber.open(file_path) as pdf:                                           # Abre el archivo PDF con pdfplumber

            dfs = []                                                                      # Inicializa una lista para almacenar los DataFrames de las tablas del PDF actual
            for page in pdf.pages:                                                        # Itera sobre cada página del PDF
                tables = page.extract_tables()                                            # Extrae las tablas de la página actual
                for table in tables:                                                      # Itera sobre cada tabla extraída
                    df = pd.DataFrame(table[1:], columns=table[0])                        # Convierte la tabla en un DataFrame de pandas
                    dfs.append(df)                                                        # Añade el DataFrame a la lista

            result = pd.concat(dfs, ignore_index=True)                                    # Concatena todos los DataFrames de las tablas del PDF actual
            all_dfs.append(result)                                                        # Añade el resultado a la lista de DataFrames

final_result = pd.concat(all_dfs, ignore_index=True)                                      # Concatena todos los DataFrames de las tablas de todos los PDF en uno solo

final_result                                                                              # Imprime el DataFrame resultante

,Expediente,Identificación,Fecha de Infracción,Matrícula,Importe,Precepto,Artículo,Puntos
0,MUL/2023/73807,14171520-R,27/10/2023,9428BXL,"100,00","Ley 18/2021, de 20/12 (LSV)","LSV, ARTÍCULO 53.1.5D",0
1,MUL/2024/5255,32415244-X,18/01/2024,4006KGS,"60,00",O.M.C. aprobada el 07/06/2018,LSV ART. 39-40/ OMORA ART. 12.1.A,0
2,MUL/2024/409,32415244-X,19/12/2023,4006KGS,"60,00",O.M.C. aprobada el 07/06/2018,LSV ART. 39-40/ OMORA ART. 12.1.C,0
3,MUL/2024/2245,32415244-X,02/01/2024,4006KGS,"60,00",O.M.C. aprobada el 07/06/2018,LSV ART. 39-40/ OMORA ART. 12.1.A,0
4,MUL/2024/4793,32421092-Q,23/01/2024,2124CPB,"200,00",O.M.C. aprobada el 07/06/2018,OMC ART. 81,0
...,...,...,...,...,...,...,...,...
1353,MUL/2023/71578,Y5969204-M,16/12/2023,0837CLK,"200,00",O.M.C. aprobada el 07/06/2018,LSV ART. 39-40/ OMC ART. 43.2.A,0
1354,MUL/2024/40,Y6387304-B,02/01/2024,C5849BV,"100,00",O.M.C. aprobada el 07/06/2018,LSV ART. 39-40/ OMC ART. 43.2.B,0
1355,MUL/2024/6750,Y7209434-Y,10/01/2024,OU6064W,"100,00","Ley 18/2021, de 20/12 (LSV)","LSV, Artículo 53",0
1356,MUL/2023/72782,Y8167543-G,13/12/2023,M2189YH,"60,00",O.M.C. aprobada el 07/06/2018,LSV ART. 39-40/ OMORA ART. 12.1.A,0
